# Import functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


from sklearn.feature_selection import mutual_info_regression

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('../../nn_physical_concepts')

from scinet import *
import scinet.ed_oscillator as edo

import tqdm

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times'],'size' : 16})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

from sklearn.model_selection import train_test_split

# Load data

In [ ]:
import pandas as pd

#extract generated data
df = pd.read_csv('../data/transient_data_ch41step.csv')

#extract forward rates
df_rates = pd.read_csv('../data/constant_data_ch41step.csv')
kf = np.zeros((1,2001))
for i in range(2001):
    kf[0,i] = float(df_rates['kf'][i][1:-1])
kf = kf[0]

# Helper functions

In [ ]:
#converts data from pandas format to scinet format with 3 observations
def my_load_traindata_3(df,thalf = 1.25e-5):
    mydf = df[df['t [s]'] <=thalf]
    in1 = np.zeros((2001*25,25))
    in2 = np.zeros((2001*25,3))
    out = np.zeros((2001*25,1))
    for j in range(25):
        for i in range(1000,3001):
            in1[(i-1000)*25 + j,:] = mydf[mydf['T [K]']==i]['Yco2']
    in2[:,0] = mydf['t [s]']
    in2[:,1] = mydf['Yo2']
    in2[:,2] = mydf['Ych4']
    out[:,0] = mydf['Yco2']
    return [in1, in2, out]
#converts data from pandas format to scinet format with 3 observations at a specific T
def my_load_testdata3(df,T,thalf = 1.25e-5 ):
    mydf = df[df['t [s]'] <= thalf]
    in1 = np.zeros((51,25))
    in2 = np.zeros((51,3))
    for i in range(51): 
        in1[i,:] = mydf[mydf['T [K]']==T]['Yco2']
        in2[i,0] = i*5e-7
    in2[:,1] = df[df['T [K]']==T]['Yo2']
    in2[:,2] = df[df['T [K]']==T]['Ych4']
    out = np.zeros((51,1))
    return [in1,in2,out]

#randomizes and splits data to train test and dev sets
def my_train_test_split(train_data3,test_size = 0.25,random_state=42):
    in1train, in1tmp = train_test_split(train_data3[0], test_size=test_size, random_state=random_state)
    in2train, in2tmp = train_test_split(train_data3[1], test_size=test_size, random_state=random_state)
    outtrain, outtmp = train_test_split(train_data3[2], test_size=test_size, random_state=random_state)
    in1test, in1dev = train_test_split(in1tmp, test_size=0.5, random_state=random_state)
    in2test, in2dev = train_test_split(in2tmp, test_size=0.5, random_state=random_state)
    outtest, outdev = train_test_split(outtmp, test_size=0.5, random_state=random_state)
    return [in1train,in2train,outtrain],[in1test,in2test,outtest],[in1dev,in2dev,outdev]

#randomizes and splits temperature to train test and dev sets
def my_temp_train_test_split(test_size = 0.25,random_state=42):
    temp = np.arange(1000,3001)
    temptrain, temptmp = train_test_split(temp, test_size=test_size, random_state=random_state)
    temptest, tempdev = train_test_split(temptmp, test_size=0.5, random_state=random_state)
    return temptrain, temptest, tempdev


#converts data from pandas format to scinet format with 3 observations at a single T and time
def my_load_testdata_singletime(df,T,time,thalf = 1.25e-5 ):
    mydf = df[df['t [s]'] <= thalf]
    in1 = np.zeros((1,25))
    in2 = np.zeros((1,3))
    in1[0,:] = mydf[mydf['T [K]']==T]['Yco2']
    in2[0,0] = time
    in2[:,1] = df[(df['T [K]']==T) & (df['t [s]']<(time+4e-7))& (df['t [s]']>(time-4e-7))]['Yo2']
    in2[:,2] = df[(df['T [K]']==T) & (df['t [s]']<(time+4e-7))& (df['t [s]']>(time-4e-7))]['Ych4']
    out = np.zeros((1,1))
    return [in1,in2,out]







# Train and predict

In [ ]:
#convert pandas to scinet input
train_data3  = my_load_traindata_3(df,thalf = 1.25e-5) 

In [ ]:
#split data
td, vd, test_data  = my_train_test_split(train_data3) #randomise and split train and test sets

In [ ]:
#initialize net
net = nn.Network(latent_size = 3,input_size =  25,input2_size = 3,output_size = 1,encoder_num_units= [500, 100],name = 'Prod1') #initialize net

In [ ]:
#initialize training

train_losses = []
dev_losses = []

#Training program
#procedure summary: 1000 epochs with alpha 1e-3, batch 512; 500 epochs with alpha 1e-4 batch 1024, 500 epochs with alpha 1e-5 batch 1024
num_phases = 3
all_epochs         = [100, 100,  100 ]
all_batches        = [32,  32, 32]
all_learning_rates = [1e-3, 1e-4, 1e-5]

myBeta = 1e-3

In [ ]:
# Train
print_frequency = 0.1

for j in tqdm.tqdm_notebook(range(num_phases)):
    num_epochs = all_epochs[j]
    batch_size = all_batches[j]
    learning_rate = all_learning_rates[j]
    check_epochs = int(print_frequency * num_epochs)
    
    for i in tqdm.tqdm_notebook(range(num_epochs)):
        net.train(1, batch_size, learning_rate, td, vd, beta_fun=(lambda x: myBeta), test_step=10 )

        # Check progress. It is recommended to use Tensorboard instead for this.
        train_recon_error = net.run(td, net.recon_loss)
        train_kl_loss     = net.run(td, net.kl_loss)
        train_loss        = train_recon_error + myBeta*train_kl_loss

        dev_recon_error   = net.run(vd, net.recon_loss)
        dev_kl_loss       = net.run(vd, net.kl_loss)
        dev_loss          = dev_recon_error + myBeta*dev_kl_loss

        train_losses.append(train_loss)
        dev_losses.append(dev_loss)

        if i%check_epochs == 0:
            print("Training: (loss, reconstruction error, kl loss): ({:.2e}, {:.2e}, {:.2e})".format(
            train_loss, train_recon_error, train_kl_loss))
            print("Dev:      (loss, reconstruction error, kl loss): ({:.2e}, {:.2e}, {:.2e})".format(
            dev_loss, dev_recon_error, dev_kl_loss))
            print("=======================================")

    print("{} epochs trained so far".format(net.tot_epochs) )

# Postprocess Results

In [ ]:
#Loss vs Epoch

plt.semilogy(range(300),train_losses,'b',label='training loss')
plt.semilogy(range(300),dev_losses,'r',label='dev loss')
plt.semilogy(range(300),abs(np.array(train_losses)-np.array(dev_losses)),'g',label='difference')
plt.legend()
plt.xlim(0,300)
plt.xlabel(r'\# Epochs')

In [ ]:
#Predictions for test set
t = np.arange(51)*5e-1 #mus
T = 1000
plt.plot([12.5,12.5],[0,0.6],'k--')

tT, vt, Ttest  = my_temp_train_test_split()

for T in Ttest[::25]: 
    test_data = my_load_testdata3(df,T)
    a_precicted = net.run(test_data, net.output).ravel()
    plt.plot(t,a_precicted,label = r'T = '+str(T))

    plt.plot(t,df[df['T [K]']==T]['Yco2'],'k--', dashes= (1,10),label='_nolegend_',linewidth = 2)

plt.plot([-1],[-1],'k',label ='Truth')

plt.plot(-1,-1,'k.',label='Predicted')

plt.legend(loc='center right',bbox_to_anchor = (1.4,0.5))
plt.xlim((0,25))
plt.ylim((0,0.6))
plt.text(3,0.55,'Observation data')
plt.text(15.5,0.55,'Answer data')
plt.ylabel(r'$Y_{\textrm{CO2}}$')
plt.xlabel(r'Time [$\mu$s]')

In [ ]:
#Interpretable temperatures

t = np.arange(51)*5e-1 #mus
T = 1000

Ttest = [1000,1200,1400,1800,2000,3000]

for T in Ttest: 
    test_data = my_load_testdata3(df,T)
    a_precicted = net.run(test_data, net.output).ravel()
    plt.plot(t,a_precicted,label = r'T = '+str(T))
    
plt.fill_between(t, df[df['T [K]']==1400]['Yco2'],  df[df['T [K]']==1800]['Yco2'],color='lightblue')
plt.text(10,0.3,'Interpretable region')
plt.legend(loc='center right',bbox_to_anchor = (1.4,0.5))
plt.xlim((0,25))
plt.ylim((0,0.6))
plt.ylabel(r'$Y_{\textrm{CO2}}$')
plt.xlabel(r'Time [$\mu$s]')
plt.savefig('chem_interp_co2.png',dpi=400,bbox_inches='tight')

In [ ]:
#evaluate latent space
latentspace = np.zeros((101,3))
time = 1.5e-5
for T in range(1000,3001,20):
    latentspace[(T-1000)/20,:] = np.array([net.run(my_load_testdata_singletime(df,T,time), net.mu)[0] ])

In [ ]:
#Interpretable latent space


plt.fill_between([1400,1800],[-15e-6,-15e-6],[17e-6,17e-6],color='lightblue')

plt.plot(range(1000,3001,20),latentspace[:,0],label = 'Neuron 1')
plt.plot(range(1000,3001,20),latentspace[:,1],label ='Neuron 2')
plt.plot(range(1000,3001,20),latentspace[:,2],label ='Neuron 3')

plt.ylabel('Latent activation')
plt.plot([1400,1400],[17e-6,-15e-6],'k--')
plt.plot([1800,1800],[17e-6,-15e-6],'k--')
plt.ylim([-15e-6,17e-6])
plt.legend()

plt.xlabel(r'$T$ [K]')
plt.xlim([1000,3000])
# plt.xticks([0,5e5,1e6,1.5e6,2e6])
plt.savefig('chem_interp_latent.png',dpi=400,bbox_inches='tight')

In [ ]:
#latent space vs T

plt.plot(range(1000,3001,20),latentspace[:,0],label = 'Neuron 1')
plt.plot(range(1000,3001,20),latentspace[:,1],label ='Neuron 2')
plt.plot(range(1000,3001,20),latentspace[:,2],label ='Neuron 3')

plt.ylabel('Latent activation')

plt.ylim([-15e-6,17e-6])
plt.legend()

plt.xlabel(r'$T$ [K]')
plt.xlim([1000,3000])
# plt.xticks([0,5e5,1e6,1.5e6,2e6])
plt.savefig('chem_interp_latent.png',dpi=400,bbox_inches='tight')

In [ ]:
#latent space vs kf

plt.plot(kf[0:2001:20],latentspace[:,0],label = 'Neuron 1')
plt.plot(kf[0:2001:20],latentspace[:,1],label ='Neuron 2')
plt.plot(kf[0:2001:20],latentspace[:,2],label = 'Neuron 3')
plt.ylabel('Latent activation')
plt.legend()
plt.xlabel(r'$k_f$')
plt.xlim([kf[400],kf[700]])
plt.ylim([-6e-7,6e-7])
plt.savefig('chem_latent.png',dpi=400,bbox_inches='tight')